### Universites anaylsis in Marsielle, France. Using data from FOURSQUARE public API.


First is importing the libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd 

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library



Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

### creating the pandas Dataframe for the universities

here i used a website to find the best univesities in marseille and then google maps to get the coordinates

In [3]:
# Creating an empty Dataframe with column names only
uni_df = pd.DataFrame(columns=['university', 'lat', 'long'])

In [4]:
# Append the data to the Dataframe by adding dictionaries

uni_df = uni_df.append({'university': 'St. Jerome', 'lat': 43.337078, 'long': 5.411026}, ignore_index=True)
uni_df = uni_df.append({'university': 'Aix-Marseille University', 'lat': 43.293816, 'long': 5.357989}, ignore_index=True)
uni_df = uni_df.append({'university': 'University of the Mediterranean', 'lat': 43.313617, 'long': 5.365699}, ignore_index=True)
uni_df = uni_df.append({'university': 'Luminy', 'lat': 43.232048, 'long': 5.439954}, ignore_index=True)
uni_df = uni_df.append({'university': 'University of Provence', 'lat': 43.316115, 'long': 5.396967}, ignore_index=True)

uni_df.head()

,university,lat,long
0,St. Jerome,43.337078,5.411026
1,Aix-Marseille University,43.293816,5.357989
2,University of the Mediterranean,43.313617,5.365699
3,Luminy,43.232048,5.439954
4,University of Provence,43.316115,5.396967


now we have to get the coordinates of marseille using geopy

In [5]:
address = 'Marseille, France'

geolocator = Nominatim(user_agent="marseille_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Marseille, France are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Marseille, France are 43.2961743, 5.3699525.


### ok now we have all that we need to draw the map of marseille and the universities to see how the are distributed

In [13]:
# create map of marseille using latitude and longitude values
map_marseille = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, uni in zip(uni_df['lat'], uni_df['long'], uni_df['university']):
    label = '{}'.format(uni)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_marseille)  

map_marseille

as we can see that the universities are far away from each other wich mean their venues can't be accessed from one univesity to the other

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

 we set the radius to 500m because it takes a healthy person about 5 minutes to walk it

In [14]:
# Define Foursquare Credentials and Version
CLIENT_ID = '2HL0APKOIE1ZKM1VCNXR3SFBS5CHABE0UI2CMKW5PFH3NZLE' # your Foursquare ID
CLIENT_SECRET = 'TG11Y0UVQM2Q1OFMDHKXRNWAYUYMGQHYMEQ1GWNCDQ5NPF53' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define a small radius so that all the venues can be accessed by less than 5 min walk

Your credentails:
CLIENT_ID: 2HL0APKOIE1ZKM1VCNXR3SFBS5CHABE0UI2CMKW5PFH3NZLE
CLIENT_SECRET:TG11Y0UVQM2Q1OFMDHKXRNWAYUYMGQHYMEQ1GWNCDQ5NPF53


### this function will get all the near by venues of all the data set

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,  
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['university',  
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### now we will run the function

In [19]:

uni_venues = getNearbyVenues(names=uni_df['university'],
                                   latitudes=uni_df['lat'],
                                   longitudes=uni_df['long']
                                  )



St. Jerome
Aix-Marseille University
University of the Mediterranean
Luminy
University of Provence


Let's check how many venues were returned for each university

In [20]:
uni_venues.groupby('university').count()

,Venue,Venue Latitude,Venue Longitude,Venue Category
university,,,,
Aix-Marseille University,30,30,30,30
Luminy,4,4,4,4
St. Jerome,4,4,4,4
University of Provence,2,2,2,2
University of the Mediterranean,7,7,7,7


unfortunately we didn't got good amount of venues in a 5 minutes walk, so we will try bigger radius like 10 minutes walk.

In [23]:
uni_venues = getNearbyVenues(names=uni_df['university'],
                                   latitudes=uni_df['lat'],
                                   longitudes=uni_df['long'],
                                    radius=1000
                                  )

uni_venues.groupby('university').count()

St. Jerome
Aix-Marseille University
University of the Mediterranean
Luminy
University of Provence


,Venue,Venue Latitude,Venue Longitude,Venue Category
university,,,,
Aix-Marseille University,64,64,64,64
Luminy,5,5,5,5
St. Jerome,7,7,7,7
University of Provence,17,17,17,17
University of the Mediterranean,32,32,32,32


that's better

#### Let's check the size of the resulting dataframe

In [24]:
print(uni_venues.shape)
uni_venues.head()

(125, 5)


,university,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. Jerome,Gymnase Iut St Jerome - Badminton,43.338496,5.410101,Badminton Court
1,St. Jerome,Corep Marseille St-Jérôme,43.334591,5.411013,Print Shop
2,St. Jerome,O'Pépé Café,43.334673,5.410106,Snack Place
3,St. Jerome,Les Terrasses De Saint Mitre,43.342412,5.416128,French Restaurant
4,St. Jerome,Total Access,43.338488,5.420348,Gas Station


### Now itls time to analyze each university

In [26]:
# one hot encoding
uni_onehot = pd.get_dummies(uni_venues[['Venue Category']], prefix="", prefix_sep="")

# add university column back to dataframe
uni_onehot['university'] = uni_venues['university'] 

# move university column to the first column
fixed_columns = [uni_onehot.columns[-1]] + list(uni_onehot.columns[:-1])
uni_onehot = uni_onehot[fixed_columns]

uni_onehot.head()

,university,Art Museum,Auto Workshop,Badminton Court,Bakery,Bar,Beach,Bistro,Buffet,Bus Stop,Business Service,Café,Church,Clothing Store,Coffee Shop,College Track,Concert Hall,Cosmetics Shop,Cultural Center,Cupcake Shop,Diner,Electronics Store,French Restaurant,Garden,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Lighthouse,Lounge,Mediterranean Restaurant,Metro Station,Moroccan Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Other Repair Shop,Park,Pedestrian Plaza,Pier,Plaza,Pool,Print Shop,Provençal Restaurant,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Trail,Train Station,Tram Station
0,St. Jerome,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. Jerome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. Jerome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,St. Jerome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,St. Jerome,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [28]:
uni_onehot.shape

(125, 72)

#### Let's find out how many unique categories can be curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(uni_venues['Venue Category'].unique())))

There are 71 uniques categories.


#### Next, let's group rows by Post code and by taking the mean of the frequency of occurrence of each category

In [30]:
uni_grouped = uni_onehot.groupby('university').mean().reset_index()
uni_grouped

,university,Art Museum,Auto Workshop,Badminton Court,Bakery,Bar,Beach,Bistro,Buffet,Bus Stop,Business Service,Café,Church,Clothing Store,Coffee Shop,College Track,Concert Hall,Cosmetics Shop,Cultural Center,Cupcake Shop,Diner,Electronics Store,French Restaurant,Garden,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Lighthouse,Lounge,Mediterranean Restaurant,Metro Station,Moroccan Restaurant,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Other Repair Shop,Park,Pedestrian Plaza,Pier,Plaza,Pool,Print Shop,Provençal Restaurant,Restaurant,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Spa,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Trail,Train Station,Tram Station
0,Aix-Marseille University,0.015625,0.000000,0.000000,0.03125,0.03125,0.015625,0.015625,0.015625,0.015625,0.000000,0.046875,0.03125,0.0000,0.00000,0.0,0.000000,0.015625,0.000000,0.00000,0.015625,0.00000,0.093750,0.015625,0.000000,0.015625,0.015625,0.015625,0.015625,0.00000,0.000000,0.0625,0.015625,0.046875,0.03125,0.00000,0.015625,0.015625,0.015625,0.03125,0.000000,0.015625,0.00000,0.015625,0.000000,0.000000,0.015625,0.000000,0.0,0.015625,0.00000,0.06250,0.015625,0.000000,0.046875,0.046875,0.000000,0.00000,0.03125,0.0000,0.000000,0.015625,0.00000,0.015625,0.00000,0.000000,0.00000,0.015625,0.03125,0.0,0.00000,0.00000
1,Luminy,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00000,0.0000,0.00000,0.2,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.200000,0.00000,0.00000,0.0000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.2,0.00000,0.00000
2,St. Jerome,0.000000,0.000000,0.142857,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.00000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000,0.00000,0.000000,0.142857,0.000000,0.000000,0.000000,0.00000,0.00000,0.0000,0.142857,0.000000,0.00000,0.000000,0.00000,0.285714,0.00000,0.000000,0.00000,0.0,0.00000,0.00000
3,University of Provence,0.000000,0.058824,0.000000,0.00000,0.00000,0.000000,0.058824,0.000000,0.000000,0.058824,0.000000,0.00000,0.0000,0.00000,0.0,0.058824,0.000000,0.058824,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.00000,0.058824,0.0000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.117647,0.000000,0.00000,0.000000,0.117647,0.058824,0.000000,0.058824,0.0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.058824,0.00000,0.00000,0.0000,0.058824,0.000000,0.00000,0.000000,0.00000,0.117647,0.00000,0.000000,0.00000,0.0,0.00000,0.00000
4,University of the Mediterranean,0.031250,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.00000,0.0625,0.03125,0.0,0.031250,0.000000,0.000000,0.03125,0.000000,0.03125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03125,0.000000,0.0000,0.000000,0.156250,0.00000,0.03125,0.000000,0.000000,0.031250,0.00000,0.062500,0.000000,0.03125,0.000000,0.000000,0.000000,0.031250,0.000000,0.0,0.000000,0.03125,0.03125,0.000000,0.000000,0.000000,0.000000,0.000000,0.03125,0.03125,0.0625,0.000000,0.000000,0.03125,0.000000,0.03125,0.000000,0.03125,0.031250,0.00000,0.0,0.03125,0.03125


#### Let's confirm the new size

In [31]:
uni_grouped.shape

(5, 72)

#### Let's print each unversity along with the top 5 most common venues

In [32]:
num_top_venues = 5

for uni in uni_grouped['university']:
    print("----"+uni+"----")
    temp = uni_grouped[uni_grouped['university'] == uni].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aix-Marseille University----
                  venue  freq
0     French Restaurant  0.09
1         Historic Site  0.06
2                 Plaza  0.06
3                 Hotel  0.05
4  Provençal Restaurant  0.05


----Luminy----
            venue  freq
0  Sandwich Place   0.2
1           Trail   0.2
2   College Track   0.2
3        Bus Stop   0.2
4            Park   0.2


----St. Jerome----
               venue  freq
0        Supermarket  0.29
1  French Restaurant  0.14
2    Badminton Court  0.14
3        Gas Station  0.14
4        Snack Place  0.14


----University of Provence----
               venue  freq
0        Music Venue  0.12
1        Supermarket  0.12
2      Metro Station  0.12
3  Other Repair Shop  0.06
4     Sandwich Place  0.06


----University of the Mediterranean----
            venue  freq
0           Hotel  0.16
1   Metro Station  0.06
2   Shopping Mall  0.06
3  Clothing Store  0.06
4      Art Museum  0.03




Now let's create the new dataframe and display the top 10 venues for each neighborhood.

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['university']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
uni_venues_sorted = pd.DataFrame(columns=columns)
uni_venues_sorted['university'] = uni_grouped['university']

for ind in np.arange(uni_grouped.shape[0]):
    uni_venues_sorted.iloc[ind, 1:] = return_most_common_venues(uni_grouped.iloc[ind, :], num_top_venues)

uni_venues_sorted.head()

,university,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aix-Marseille University,French Restaurant,Plaza,Historic Site,Café,Provençal Restaurant,Restaurant,Hotel,Seafood Restaurant,Church,Mediterranean Restaurant
1,Luminy,Bus Stop,Trail,Park,College Track,Sandwich Place,Gastropub,Electronics Store,French Restaurant,Garden,Gas Station
2,St. Jerome,Supermarket,Print Shop,Snack Place,Badminton Court,French Restaurant,Gas Station,Cupcake Shop,Diner,Electronics Store,Garden
3,University of Provence,Music Venue,Supermarket,Metro Station,Concert Hall,Business Service,Cultural Center,Neighborhood,Sandwich Place,Snack Place,Bistro
4,University of the Mediterranean,Hotel,Clothing Store,Metro Station,Shopping Mall,Italian Restaurant,Nightclub,Multiplex,Lounge,Train Station,Tram Station


## Now Cluster 


Run *k*-means to cluster the universities into 3 clusters.

In [39]:
# set number of clusters
kclusters = 3

uni_grouped_clustering = uni_grouped.drop('university', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(uni_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each university.

In [40]:
# add clustering labels
uni_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

uni_merged = uni_df

# merge uni_grouped with uni_data to add latitude/longitude for each university
uni_merged = uni_merged.join(uni_venues_sorted.set_index('university'), on='university')



uni_merged.head() # check the last columns!

,university,lat,long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,St. Jerome,43.337078,5.411026,1,Supermarket,Print Shop,Snack Place,Badminton Court,French Restaurant,Gas Station,Cupcake Shop,Diner,Electronics Store,Garden
1,Aix-Marseille University,43.293816,5.357989,0,French Restaurant,Plaza,Historic Site,Café,Provençal Restaurant,Restaurant,Hotel,Seafood Restaurant,Church,Mediterranean Restaurant
2,University of the Mediterranean,43.313617,5.365699,0,Hotel,Clothing Store,Metro Station,Shopping Mall,Italian Restaurant,Nightclub,Multiplex,Lounge,Train Station,Tram Station
3,Luminy,43.232048,5.439954,2,Bus Stop,Trail,Park,College Track,Sandwich Place,Gastropub,Electronics Store,French Restaurant,Garden,Gas Station
4,University of Provence,43.316115,5.396967,0,Music Venue,Supermarket,Metro Station,Concert Hall,Business Service,Cultural Center,Neighborhood,Sandwich Place,Snack Place,Bistro


In [45]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(uni_merged['lat'], uni_merged['long'], uni_merged['university'], uni_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

well now after clustering and ploting the universities it's became obvious that the universities in cluster 0 are the bes in term of their position in marseille for the student because the have a lot of venues near by in less than a 10 minuets walk 